In [1]:
from pyspark.sql import SparkSession 
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [2]:
spark = SparkSession.builder.appName("Logistic regression model").getOrCreate()

In [6]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [8]:
data.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|  Cynthia Norton|37.0|    

In [10]:
data.describe().show(2)

+-------+-----+-----------------+-----------------+------------------+----------------+-----------------+--------+-------+-------------------+
|summary|Names|              Age|   Total_Purchase|   Account_Manager|           Years|        Num_Sites|Location|Company|              Churn|
+-------+-----+-----------------+-----------------+------------------+----------------+-----------------+--------+-------+-------------------+
|  count|  900|              900|              900|               900|             900|              900|     900|    900|                900|
|   mean| null|41.81666666666667|10062.82403333334|0.4811111111111111|5.27315555555555|8.587777777777777|    null|   null|0.16666666666666666|
+-------+-----+-----------------+-----------------+------------------+----------------+-----------------+--------+-------+-------------------+
only showing top 2 rows



In [11]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [14]:
output = assembler.transform(data)

In [15]:
final_data = output.select('features','churn')

In [16]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [17]:
from pyspark.ml.classification import LogisticRegression

In [18]:
lr_churn = LogisticRegression(labelCol='churn')

In [19]:
fitted_churn_model = lr_churn.fit(train_churn)

In [20]:
training_sum = fitted_churn_model.summary

In [21]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                629|                629|
|   mean| 0.1589825119236884|0.11128775834658187|
| stddev|0.36595079201684555| 0.3147384338033101|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [25]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[4.76595202500676...|[0.99155711122365...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.94698096738033...|[0.95012060641417...|       0.0|
|[29.0,13255.05,1....|    0|[4.15107186830498...|[0.98449661188182...|       0.0|
|[30.0,10744.14,1....|    1|[2.10514626053037...|[0.89140236277256...|       0.0|
|[30.0,13473.35,0....|    0|[2.58869621002672...|[0.93013053428328...|       0.0|
|[31.0,7073.61,0.0...|    0|[3.25484284468922...|[0.96284674406809...|       0.0|
|[31.0,8829.83,1.0...|    0|[4.45698839724346...|[0.98853571698817...|       0.0|
|[31.0,10182.6,1.0...|    0|[4.69648690559423...|[0.99095526799377...|       0.0|
|[31.0,11743.24,0....|    0|[6.58037020561393...|[0.99861458643323...|       0.0|
|[32.0,7896.65,0

In [26]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [27]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [28]:
auc

0.7741628959276018

In [29]:
# predict on new data

In [30]:
final_lr_model = lr_churn.fit(final_data)

In [32]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [33]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [34]:
test_new_customers = assembler.transform(new_customers)

In [35]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [36]:
final_results = final_lr_model.transform(test_new_customers)

In [39]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [41]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          